# Investigação da consistência dos dados

Ao iniciar uma análise das despesas da pandemia do COVID-19 reparamos que os dados
tinham alguns problemas. Algumas entradas estavam duplicadas e outras não batiam.

Essa análise visa comparar os dados apresentados no Tribunal de Contas dos Municípios da Bahia
(TCM-BA) e o Portal da Transparência da Prefeitura de Feira de Santana.

## Dados utilizados

Para avaliar os dados escolhemos o mês de Julho de 2020 (escolhemos aleatoriamente mas poderia ser qualquer um)
e selecionamos os **pagamentos** da **Prefeitura Municipal de Feira de Santana**.

* [Portal da Transparência da Prefeitura de Feira de Santana](http://www.transparencia.feiradesantana.ba.gov.br/index.php?view=despesa)
* [Tribunal de Contas dos Municípios da Bahia](https://www.tcm.ba.gov.br/consulta-de-despesas)

Exportamos esses dados para XLS e CSV, respectivamente.

## Vamos a análise!

In [6]:
import pandas as pd


cityhall_df = pd.read_excel("../prefeitura-despesas-prefeitura-julho-2020.csv.xls")
tcm_df = pd.read_json("../tcmba-despesas-prefeitura-julho-2020.json")

Uma amostra dos dados da prefeitura:

In [10]:
cityhall_df.sample(3)

,Data de Publicacao,Fase,Numero,N do processo,Bem / Servico Prestado,Credor,CPF / CNPJ,Valor,Funcao,Subfuncao,Natureza,Fonte,Modalidade
1828,10/07/2020,PAGAMENTO,6557,,REF SERVICOS DE FORNECIMENTO DE AGUA P/ O ZOON...,EMP BAIANA DE AGUA E SANEAMENTO - EMBASA ...,13.504.675/0001-10,"1.671,53",10 - SAUDE ...,305 - VIGILIANCIA EPIDEMIOLOGICA ...,339039110000000000 - Servicos de Fornecimnto d...,0002 - REC.IMP.TRANSF.IMP.SAUDE 15 ...,Isento
1827,10/07/2020,PAGAMENTO,6570,,REF SERVICOS DE FORNECIMENTO DE AGUA P/ OS PSF...,EMP BAIANA DE AGUA E SANEAMENTO - EMBASA ...,13.504.675/0001-10,"1.562,40",10 - SAUDE ...,301 - ATENCAO BASICA ...,339039110000000000 - Servicos de Fornecimnto d...,0002 - REC.IMP.TRANSF.IMP.SAUDE 15 ...,Isento
2166,07/07/2020,PAGAMENTO,07989-20,,REF A DESPESA COM CONSUMO DE ENERGIA ELETRICA....,COELBA ...,15.139.629/0001-94,"1.216,13",04 - ADMINISTRACAO ...,122 - ADMINISTRACAO GERAL ...,339039100000 - Serviços de Energia Elétrica ...,0000 - RECURSOS ORDINARIOS ...,ISENTO


In [17]:
cityhall_df.shape

(2721, 13)

Uma amostra dos dados do TCM-BA:

In [9]:
tcm_df.sample(3)

,NumeroDocumeto,empenho,data,municipio,unidade,cd_Unidade,orgao,unidadeOrcamentaria,credor,valor,cd_Orgao,cd_UnidadeOrcamentaria,dataEmpenho
102,07613-20-PMFS,20000148,2020-07-02 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SEC.DESENVOLVIMENTO SOCIAL,SEC.DESENVOLVIMENTO SOCIAL,M E B SERVICOS E LOCAÇÕES LTDA.,189.17,12,1212,2020-02-05 00:00:00.000
362,05827-20-FMS,20000020,2020-07-03 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,GEISELL MARTINEZ TORRES,1571.00,11,1111,2020-01-02 00:00:00.000
154,07744-20-PMFS,20000019,2020-07-02 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SEC.DESENVOLVIMENTO SOCIAL,SEC.DESENVOLVIMENTO SOCIAL,DIVALDE BEZERRA DA SILVA,2455.30,12,1212,2020-01-02 00:00:00.000


In [18]:
tcm_df.shape

(2143, 13)

A partir do número de linhas podemos ver que os números não batem.
Mas vamos tentar cruzar os códigos de empenho para ver o que encontramos.

Antes de começarmos a explorar um pouco mais, vamos mapear os campos de cada arquivo:

| Campo (o que é) | Prefeitura | TCM-BA        | Observações |
| ------------- | ------------- | ------------- | ------------- |
| Data | `Data de Publicacao` | `data` | Assumindo que essa é a data do pagamento e não a data do empenho |
| Fase da despesa | `Fase` |  | |
| Número da despesa | `Numero` | `empenho` | |
| Número do processo | `N do processo` | `NumeroDocumeto` | Necessário confirmar |
| Descrição da despesa | `Bem / Servico Prestado` |  | |
| Credor | `Credor` | `credor` | |
| CPF/CNPJ do credor | `CPF / CNPJ` | | Presente apenas na página de detalhes |
| Valor | `Valor` | `valor` | |
| Função | `Funcao` | | |
| Subfunção | `Subfuncao` | | |
| Natureza | `Natureza` | | |
| Fonte | `Fonte` | | |
| Modalidade | `Modalidade` |  | |


## Match dos pagamentos

Infelizmente não temos tantos campos assim para fazer o _match_ das despesas.
Mas podemos tentar a partir do número da despesa e o número do processo.

O número da despesa, representado por `NumeroDocumeto` no TCM-BA, vem seguido do órgão.
Exemplo: `07613-20-PMFS`. Precisamos cortar essa parte para bater com o número nos dados da prefeitura.

In [27]:
cityhall_df["Numero"]

0       09149-20                      
1       09150-20                      
2       3107                          
3       3106                          
4       3105                          
                     ...              
2716    08717-20                      
2717    08715-20                      
2718    08888-20                      
2719    08889-20                      
2720    09910-20                      
Name: Numero, Length: 2721, dtype: object

In [24]:
tcm_df["NumeroDocumeto"]

0        05705-20-FMS
1        05734-20-FMS
2        05716-20-FMS
3        05744-20-FMS
4        05714-20-FMS
            ...      
2138    09900-20-PMFS
2139    09901-20-PMFS
2140     06848-20-FMS
2141     06847-20-FMS
2142     06844-20-FMS
Name: NumeroDocumeto, Length: 2143, dtype: object